In [1]:
import nltk
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
class TextSimilarityExample: 
    def __init__(self): 
        self.statements = [ # 유사도를 찾고자 하는 샘플 문장 정의
            'ruled india',
            'Chalukyas ruled Badami',
            'So many kingdoms ruled India',
            'Lalbagh is a botanical garden in India'
        ]
        
    def TF(self, sentence): # 단어의 TF 계산
        words = nltk.word_tokenize(sentence.lower()) # 문장을 소문자로 변환 후 모든 단어 추출
        freq = nltk.FreqDist(words) # 단어의 빈도수 계산
        dictionary = {}
        
        for key in freq.keys():
            norm = freq[key] / float(len(words)) # 각 단어 빈도수를 한 문서의 전체 단어 수로 나누기(표준화된 빈도수 계산)
            dictionary[key] = norm # 각 단어에 맞는 TF 저장
            
        return dictionary # tf 값이 포함된 딕셔너리 반환
    
    def IDF(self): # 단어의 IDF 계산
        def idf(TotalNumberOfDocuments, NumberOfDocumentsWithThisWord): # IDF 계산 공식 정의
            return 1.0 + math.log(TotalNumberOfDocuments / NumberOfDocumentsWithThisWord)
        
        numDocuments = len(self.statements) # 전체 문서 수
        uniqueWords = {} # 단어 구별을 위한 딕셔너리 선언
        idfValues = {} # idf 값 계산을 위한 딕셔너리 선언
        
        for sentence in self.statements: # 전체 문서에 대해 반복
            for word in set(nltk.word_tokenize(sentence.lower())): # 각 문서에 대해 단어 유무 확인
                if word not in uniqueWords: # 처음 보는 단어인 경우
                    uniqueWords[word] = 1
                else: # 다른 문서에서 있었던 단어인 경우
                    uniqueWords[word] += 1
                    
        for word in uniqueWords: # 전체 단어에 대해 반복
            idfValues[word] = idf(numDocuments, uniqueWords[word]) # 지역함수를 호출하여 IDF 계산
            
        return idfValues # IDF 값이 포함된 딕셔너리 반환
    
    def TF_IDF(self, query):
        words = nltk.word_tokenize(query.lower()) # 쿼리 문자열을 토큰으로 분리
        idf = self.IDF() # 전체 문서에 대한 IDF 계산
        vectors = {}
        
        for sentence in self.statements: 
            tf = self.TF(sentence) # 각 문장에 대해 TF 계산
            for word in words:
                tfv = tf[word] if word in tf else 0.0 # 검색 쿼리의 각 단어에 대해 TF 계산
                idfv = idf[word] if word in idf else 0.0 # 검색 쿼리의 각 단어에 대해 IDF 계산
                mul = tfv * idfv # TF-IDF 계산
                if word not in vectors: 
                    vectors[word] = [] 
                vectors[word].append(mul) # 검색 쿼리의 각 단어에 대한 TF-IDF 저장
                
        return vectors # 검색 쿼리의 각 단어에 대한 벡터 리스트 반환
    
    def displayVectors(self, vectors): # 벡터 내용 화면에 표시
        print(self.statements)
        for word in vectors:
            print('{} -> {}'.format(word, vectors[word]))
            
    def cosineSimilarity(self): # 코사인 유사도 계산 함수 정의
        vec = TfidfVectorizer() # vectorizer 객체 생성
        matrix = vec.fit_transform(self.statements) # 관심 있는 모든 문서에 대해 TF-IDF 행렬 작성
        for i in range(len(self.statements)):
            print('\tsimilarity of document {} with others'.format(i))
            similarity = cosine_similarity(matrix[i:i+1], matrix)
            print(similarity)
            
    def demo(self): # 이전에 정의한 모든 함수 실행
        inputQuery = self.statements[0]
        vectors = self.TF_IDF(inputQuery)
        self.displayVectors(vectors)
        self.cosineSimilarity()

In [3]:
similarity = TextSimilarityExample()
similarity.demo()

['ruled india', 'Chalukyas ruled Badami', 'So many kingdoms ruled India', 'Lalbagh is a botanical garden in India']
ruled -> [0.6438410362258904, 0.42922735748392693, 0.2575364144903562, 0.0]
india -> [0.6438410362258904, 0.0, 0.2575364144903562, 0.18395458177882582]
	similarity of document 0 with others
[[1.         0.29088811 0.46216171 0.19409143]]
	similarity of document 1 with others
[[0.29088811 1.         0.13443735 0.        ]]
	similarity of document 2 with others
[[0.46216171 0.13443735 1.         0.08970163]]
	similarity of document 3 with others
[[0.19409143 0.         0.08970163 1.        ]]
